In [23]:
import pandas as pd
import numpy as np
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import geopandas as gpd
from libpysal.weights import KNN
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm
import os

In [24]:
# Load dataset (Manually set the file path for each category)
file_path = "new_dataset\\Mammalia_top_species.csv"  # Change this for different categories
df = pd.read_csv(file_path)

# Ensure observed_date is in datetime format
df["observed_date"] = pd.to_datetime(df["observed_date"])

# Define dependent variable (species observation count)
df["species_count"] = df.groupby(["latitude", "longitude"])["scientific_name"].transform("count")
y = df["species_count"].values.reshape(-1, 1)  # Dependent variable

# Define independent variables (weather conditions)
X = df[["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max"]].values

# Create GeoDataFrame with original lat/lon
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

# Project to metric CRS (Web Mercator)
gdf = gdf.to_crs(epsg=3857)

# Extract projected coordinates
coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))


In [25]:
# Automatically find best bandwidth
bw = Sel_BW(coords, y, X, kernel="gaussian").search()
print(f"Optimal Bandwidth (in meters): {bw}")

Optimal Bandwidth (in meters): 50.0


In [26]:
# Run GWR model with selected bandwidth
gwr_model = GWR(coords, y, X, bw, kernel="gaussian")

try:
    gwr_results = gwr_model.fit()
except Exception as e:
    print(f"GWR model fitting failed: {e}")
    exit()

# Print model summary
print(gwr_results.summary())

Model type                                                         Gaussian
Number of observations:                                                5271
Number of covariates:                                                     4

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                          20240.055
Log-likelihood:                                                  -11025.141
AIC:                                                              22058.282
AICc:                                                             22060.294
BIC:                                                             -24898.006
R2:                                                                   0.000
Adj. R2:                                                             -0.000

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [27]:
# Save temperature coefficient into GeoDataFrame
gdf["temp_coeff"] = gwr_results.params[:, 0]  # Coefficient for temperature

# ------------------ CREATE INTERACTIVE MAP ------------------ #
# Reproject back to lat/lon for mapping
gdf = gdf.to_crs(epsg=4326)

# Initialize a folium map centered at the mean location
m = folium.Map(location=[gdf["geometry"].y.mean(), gdf["geometry"].x.mean()], zoom_start=5)

# Create a diverging color map (red = positive, blue = negative)
colormap = cm.linear.RdBu_11.scale(gdf["temp_coeff"].min(), gdf["temp_coeff"].max())
colormap.caption = "Temperature Coefficient"

# Add points with gradient color
for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=colormap(row["temp_coeff"]),
        fill=True,
        fill_color=colormap(row["temp_coeff"]),
        fill_opacity=0.7,
        popup=f"Temp Coeff: {row['temp_coeff']:.4f}",
    ).add_to(m)

# Add the color scale legend
m.add_child(colormap)

# Generate map filename based on category
category_name = os.path.basename(file_path).replace(".csv", "")
map_filename = f"{category_name}_gwr_temp_map_gaussian.html"
m.save(map_filename)

print(f"Map saved as {map_filename} - Open this file in your browser to view the interactive map.")

Map saved as Mammalia_top_species_gwr_temp_map_gaussian.html - Open this file in your browser to view the interactive map.


In [28]:
# # Convert results into a DataFrame
# df["precip_coeff"] = gwr_results.params[:, 1]  # Coefficients for precipitation

# # Convert to GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# # ------------------ NEW: CREATE INTERACTIVE MAP WITH GRADIENT COLORS ------------------ #
# # Initialize a folium map centered at the mean latitude and longitude
# m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)

# # Create a color scale (Black → Blue → Aqua → White)
# colormap = cm.LinearColormap(
#     colors=["Black", "Blue", "Aqua", "White"], 
#     vmin=df["precip_coeff"].min(), 
#     vmax=df["precip_coeff"].max(),
#     caption="Precipitation Coefficient"
# )

# # Add points to the map
# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=5,
#         color=colormap(row["precip_coeff"]),
#         fill=True,
#         fill_color=colormap(row["precip_coeff"]),
#         fill_opacity=0.7,
#         popup=f"Precip Coeff: {row['precip_coeff']:.4f}",
#     ).add_to(m)

# # Add the colormap legend to the map
# m.add_child(colormap)

# # Save the map
# m.save("gwr_precip_map.html")
# print("Map saved as gwr_precip_map.html - Open this file in your browser to view the interactive map.")


In [29]:
# df["wind_coeff"] = gwr_results.params[:, 2]  # Coefficients for wind speed

# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)

# colormap = cm.LinearColormap(
#     colors=["Black", "Blue", "Aqua", "White"], 
#     vmin=df["wind_coeff"].min(), 
#     vmax=df["wind_coeff"].max(),
#     caption="Wind Speed Coefficient"
# )

# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=5,
#         color=colormap(row["wind_coeff"]),
#         fill=True,
#         fill_color=colormap(row["wind_coeff"]),
#         fill_opacity=0.7,
#         popup=f"Wind Coeff: {row['wind_coeff']:.4f}",
#     ).add_to(m)

# m.add_child(colormap)

# m.save("gwr_wind_map.html")
# print("Map saved as gwr_wind_map.html - Open this file in your browser to view the interactive map.")


In [30]:
# df["snow_coeff"] = gwr_results.params[:, 3]  # Coefficients for snowfall

# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=5)

# colormap = cm.LinearColormap(
#     colors=["Black", "Blue", "Aqua", "White"], 
#     vmin=df["snow_coeff"].min(), 
#     vmax=df["snow_coeff"].max(),
#     caption="Snowfall Coefficient"
# )

# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=5,
#         color=colormap(row["snow_coeff"]),
#         fill=True,
#         fill_color=colormap(row["snow_coeff"]),
#         fill_opacity=0.7,
#         popup=f"Snowfall Coeff: {row['snow_coeff']:.4f}",
#     ).add_to(m)

# m.add_child(colormap)

# m.save("gwr_snow_map.html")
# print("Map saved as gwr_snow_map.html - Open this file in your browser to view the interactive map.")
